In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import auc, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import datetime as dt

In [357]:
target = pd.read_csv('data/target_competencia_ids.csv')
target.head()

,ref_hash,obj
0,1000169251625791246_sc,0
1,1000169251625791246_st,0
2,1000395625957344683_sc,0
3,1000395625957344683_st,0
4,1003027494996471685_sc,0


In [356]:
target = pd.read_csv('data/target_competencia_ids.csv')
target['ref_hash'] = target['ref_hash'].transform(lambda x: str(x)[:-3])
target['ref_hash'] = target['ref_hash'].astype(np.int64)
target.drop_duplicates(subset = 'ref_hash', inplace = True)
target.head()

,ref_hash,obj
0,1000169251625791246,0
2,1000395625957344683,0
4,1003027494996471685,0
6,1006670001679961544,0
8,1007573308966476713,0


In [198]:
auct1 = pd.read_csv('data/auctions_ventana6.csv', dtype = { "ref_type_id": np.int8, "source_id": np.int8})
auct2 = pd.read_csv('data/auctions_ventana7.csv', dtype = { "ref_type_id": np.int8, "source_id": np.int8})

In [199]:
auct1['date'] = pd.to_datetime(auct1['date'])
ids_list1=auct1['device_id'].unique()
auct_predict1= auct2[auct2['device_id'].isin(ids_list1)]
auct_predict1['date'] = pd.to_datetime(auct_predict1['date'])

C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [200]:

#ojo cambiar la fecha de la ventana
auct_predict1['date_inicial'] = dt.datetime(2019,4, 24)
auct_predict1['timeToAuction'] = (auct_predict1['date'] - auct_predict1['date_inicial'])/np.timedelta64(1,'s')
auction_time = auct_predict1.groupby('device_id').agg({'timeToAuction': 'min'}).reset_index()
auction_time.columns = ['ref_hash', 'st']

auction_time = target[['ref_hash']].merge(auction_time, on = 'ref_hash', how = 'outer')
# lleno los tiempos vacios con cero
auction_time['st'] = auction_time['st'].fillna((72 * 60 * 60))

auction_count = auct1.groupby('device_id').agg({'date': 'count'}).reset_index()
auction_count.columns = ['ref_hash', 'auctions_count']
auction_count = target[['ref_hash']].merge(auction_count, on = 'ref_hash', how = 'outer')
auction_count['auctions_count'] = auction_count['auctions_count'].fillna(0)

data = auction_count.merge(auction_time, on = 'ref_hash')

auct1['dia_semana'] = auct1['date'].dt.dayofweek
auct1['dia_semana']=auct1['dia_semana'] + 1
day_mean = auct1.groupby('device_id').agg({'dia_semana': 'mean'}).reset_index()
day_mean.columns = ['ref_hash', 'dia_semana_mean']

day_mean = target[['ref_hash']].merge(day_mean, on = 'ref_hash', how = 'outer')
day_mean['dia_semana_mean'] = day_mean['dia_semana_mean'].fillna(0)

data =data.merge(day_mean, on = 'ref_hash')

auct1['hora']=auct1['date'].dt.hour
hour_mean = auct1.groupby('device_id').agg({'hora': 'mean'}).reset_index()
hour_mean.columns = ['ref_hash', 'hora_mean']
hour_mean['hora_mean'] = hour_mean['hora_mean'] +1
hour_mean = target[['ref_hash']].merge(hour_mean, on = 'ref_hash', how = 'outer')
hour_mean['hora_mean'] = hour_mean['hora_mean'].fillna(0)

data =data.merge(hour_mean, on = 'ref_hash')


C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [201]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289982 entries, 0 to 289981
Data columns (total 5 columns):
ref_hash           289982 non-null int64
auctions_count     289982 non-null float64
st                 289982 non-null float64
dia_semana_mean    289982 non-null float64
hora_mean          289982 non-null float64
dtypes: float64(4), int64(1)
memory usage: 13.3 MB


In [202]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [203]:
evts1 = pd.read_csv("data/events_ventana6.csv", dtype = {"event_id": np.int16, "application_id": np.int16, 'device_countrycode': 'category', 'device_os_version': 'category', 'device_brand': 'category', 'device_model': 'category', 'device_city': 'category', 'session_user_agent': 'category', 'trans_id': 'category', 'user_agent': 'category', 'carrier' : 'category', 'kind': 'category', 'device_os': 'category', 'connection_type': 'category', 'ip_address': 'category', 'device_language': 'category'})

In [204]:
events_count = evts1.groupby('ref_hash').agg({'date': 'count'}).reset_index()
events_count.columns = ['ref_hash', 'cantidad de eventos']

events_count = target[['ref_hash']].merge(events_count, on = 'ref_hash', how = 'outer')
events_count['cantidad de eventos'] = events_count['cantidad de eventos'].fillna(0)
data =data.merge(events_count, on = 'ref_hash')
data=data[['ref_hash', 'auctions_count', 'dia_semana_mean', 'hora_mean','cantidad de eventos','st']]


In [205]:
data.head()

,ref_hash,auctions_count,dia_semana_mean,hora_mean,cantidad de eventos,st
0,1000169251625791246,7.0,2.857143,17.428571,3.0,76114.647428
1,1000395625957344683,16.0,2.875000,13.750000,0.0,8034.974209
2,1003027494996471685,168.0,3.220238,8.952381,22.0,12171.691046
3,1006670001679961544,1.0,3.000000,19.000000,0.0,64857.606340
4,1007573308966476713,7.0,3.142857,13.857143,0.0,18726.239096


In [206]:
data.isnull().sum()

ref_hash               0
auctions_count         0
dia_semana_mean        0
hora_mean              0
cantidad de eventos    0
st                     0
dtype: int64

In [207]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [208]:
#Guardo la ventana con los features
data.to_csv(path_or_buf = 'data/features1_aut6.csv', index = False)
ts1= pd.read_csv( 'data/features1_aut6.csv')
ts1.head()

,ref_hash,auctions_count,dia_semana_mean,hora_mean,cantidad de eventos,st
0,1000169251625791246,7.0,2.857143,17.428571,3.0,76114.647428
1,1000395625957344683,16.0,2.875000,13.750000,0.0,8034.974209
2,1003027494996471685,168.0,3.220238,8.952381,22.0,12171.691046
3,1006670001679961544,1.0,3.000000,19.000000,0.0,64857.606340
4,1007573308966476713,7.0,3.142857,13.857143,0.0,18726.239096


In [209]:

ts1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77527 entries, 0 to 77526
Data columns (total 6 columns):
ref_hash               77527 non-null int64
auctions_count         77527 non-null float64
dia_semana_mean        77527 non-null float64
hora_mean              77527 non-null float64
cantidad de eventos    77527 non-null float64
st                     77527 non-null float64
dtypes: float64(5), int64(1)
memory usage: 3.5 MB


# Armo los fetures del target

In [216]:
auct1 = pd.read_csv('data/auctions_ventana7.csv', dtype = { "ref_type_id": np.int8, "source_id": np.int8})
auct1['date'] = pd.to_datetime(auct1['date'])
ids_list1=target['ref_hash'].unique()
auct1= auct1[auct1['device_id'].isin(ids_list1)]
auct1['device_id'].unique().size

3351

In [218]:
auction_count = auct1.groupby('device_id').agg({'date': 'count'}).reset_index()
auction_count.columns = ['ref_hash', 'auctions_count']
#hago un merge left para que darme solo con los ids del target
auction_count = target[['ref_hash']].merge(auction_count, on = 'ref_hash', how = 'left')
auction_count['auctions_count'] = auction_count['auctions_count'].fillna(0)

data = auction_count
data['st']=0

auct1['dia_semana'] = auct1['date'].dt.dayofweek
auct1['dia_semana']=auct1['dia_semana'] + 1
day_mean = auct1.groupby('device_id').agg({'dia_semana': 'mean'}).reset_index()
day_mean.columns = ['ref_hash', 'dia_semana_mean']

day_mean = target[['ref_hash']].merge(day_mean, on = 'ref_hash', how = 'left')
day_mean['dia_semana_mean'] = day_mean['dia_semana_mean'].fillna(0)

data =data.merge(day_mean, on = 'ref_hash')

auct1['hora']=auct1['date'].dt.hour
hour_mean = auct1.groupby('device_id').agg({'hora': 'mean'}).reset_index()
hour_mean.columns = ['ref_hash', 'hora_mean']
hour_mean['hora_mean'] = hour_mean['hora_mean'] +1
hour_mean = target[['ref_hash']].merge(hour_mean, on = 'ref_hash', how = 'left')
hour_mean['hora_mean'] = hour_mean['hora_mean'].fillna(0)

data =data.merge(hour_mean, on = 'ref_hash')

In [219]:
data.head()

,ref_hash,auctions_count,st,dia_semana_mean,hora_mean
0,1000169251625791246,13.0,0,4.538462,11.923077
1,1000395625957344683,15.0,0,3.533333,15.733333
2,1003027494996471685,168.0,0,3.220238,8.952381
3,1006670001679961544,3.0,0,4.333333,19.000000
4,1007573308966476713,7.0,0,3.571429,15.857143


In [220]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [221]:
evts1 = pd.read_csv("data/events_ventana7.csv", dtype = {"event_id": np.int16, "application_id": np.int16, 'device_countrycode': 'category', 'device_os_version': 'category', 'device_brand': 'category', 'device_model': 'category', 'device_city': 'category', 'session_user_agent': 'category', 'trans_id': 'category', 'user_agent': 'category', 'carrier' : 'category', 'kind': 'category', 'device_os': 'category', 'connection_type': 'category', 'ip_address': 'category', 'device_language': 'category'})

In [222]:
events_count = evts1.groupby('ref_hash').agg({'date': 'count'}).reset_index()
events_count.columns = ['ref_hash', 'cantidad de eventos']

events_count = target[['ref_hash']].merge(events_count, on = 'ref_hash', how = 'left')
events_count['cantidad de eventos'] = events_count['cantidad de eventos'].fillna(0)
data =data.merge(events_count, on = 'ref_hash')
data=data[['ref_hash', 'auctions_count', 'dia_semana_mean', 'hora_mean','cantidad de eventos','st']]

In [223]:
data.head()

,ref_hash,auctions_count,dia_semana_mean,hora_mean,cantidad de eventos,st
0,1000169251625791246,13.0,4.538462,11.923077,2.0,0
1,1000395625957344683,15.0,3.533333,15.733333,0.0,0
2,1003027494996471685,168.0,3.220238,8.952381,9.0,0
3,1006670001679961544,3.0,4.333333,19.000000,0.0,0
4,1007573308966476713,7.0,3.571429,15.857143,0.0,0


In [224]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [225]:
#Guardo la ventana con los features
data.to_csv(path_or_buf = 'data/features1_target.csv', index = False)
ts1= pd.read_csv( 'data/features1_target.csv')
ts1.head()

,ref_hash,auctions_count,dia_semana_mean,hora_mean,cantidad de eventos,st
0,1000169251625791246,13.0,4.538462,11.923077,2.0,0
1,1000395625957344683,15.0,3.533333,15.733333,0.0,0
2,1003027494996471685,168.0,3.220238,8.952381,9.0,0
3,1006670001679961544,3.0,4.333333,19.000000,0.0,0
4,1007573308966476713,7.0,3.571429,15.857143,0.0,0


In [226]:
merged=ts1.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

# Armo los  features del installs

In [333]:
inst1 = pd.read_csv('data/installs_ventana6.csv', dtype = {'application_id': np.int16, 'ref_type': 'category', 'click_hash': 'category', 'device_country_code': 'category', 'device_brand': 'category', 'device_model': 'category', 'kind': 'category', 'device_language': 'category'})
inst2 = pd.read_csv('data/installs_ventana7.csv', dtype = {'application_id': np.int16, 'ref_type': 'category', 'click_hash': 'category', 'device_country_code': 'category', 'device_brand': 'category', 'device_model': 'category', 'kind': 'category', 'device_language': 'category'})

In [334]:
inst1['created'] = pd.to_datetime(inst1['created'])
ids_list1=inst1['ref_hash'].unique()
inst_predict1= inst2[inst2['ref_hash'].isin(ids_list1)]
inst_predict1['created'] = pd.to_datetime(inst_predict1['created'])

C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [335]:
#ojo cambiar la fecha de la ventana
inst_predict1['date_inicial'] = dt.datetime(2019,4, 24)
inst_predict1['timeToconvertion'] = (inst_predict1['created'] - inst_predict1['date_inicial'])/np.timedelta64(1,'s')
install_time = inst_predict1.groupby('ref_hash').agg({'timeToconvertion': 'min'}).reset_index()
install_time.columns = ['ref_hash', 'sc']

C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\AJorg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [336]:
install_time = target[['ref_hash']].merge(install_time, on = 'ref_hash', how = 'outer')
# lleno los tiempos vacios con cero
install_time['sc'] = install_time['sc'].fillna((72 * 60 * 60))

install_count = inst1.groupby('ref_hash').agg({'created': 'count'}).reset_index()
install_count.columns = ['ref_hash', 'installs_count']
install_count = target[['ref_hash']].merge(install_count, on = 'ref_hash', how = 'outer')
install_count['installs_count'] = install_count['installs_count'].fillna(0)

data = install_count.merge(install_time, on = 'ref_hash')

inst1['dia_semana'] = inst1['created'].dt.dayofweek
inst1['dia_semana']=inst1['dia_semana'] + 1
day_mean = inst1.groupby('ref_hash').agg({'dia_semana': 'mean'}).reset_index()
day_mean.columns = ['ref_hash', 'dia_semana_mean']

day_mean = target[['ref_hash']].merge(day_mean, on = 'ref_hash', how = 'outer')
day_mean['dia_semana_mean'] = day_mean['dia_semana_mean'].fillna(0)

data =data.merge(day_mean, on = 'ref_hash')

inst1['hora']=inst1['created'].dt.hour
hour_mean = inst1.groupby('ref_hash').agg({'hora': 'mean'}).reset_index()
hour_mean.columns = ['ref_hash', 'hora_mean']
hour_mean['hora_mean'] = hour_mean['hora_mean'] + 1
hour_mean = target[['ref_hash']].merge(hour_mean, on = 'ref_hash', how = 'outer')
hour_mean['hora_mean'] = hour_mean['hora_mean'].fillna(0)

data =data.merge(hour_mean, on = 'ref_hash')


In [337]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [237]:
merged=auction_time.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [338]:
evts1 = pd.read_csv("data/events_ventana6.csv", dtype = {"event_id": np.int16, "application_id": np.int16, 'device_countrycode': 'category', 'device_os_version': 'category', 'device_brand': 'category', 'device_model': 'category', 'device_city': 'category', 'session_user_agent': 'category', 'trans_id': 'category', 'user_agent': 'category', 'carrier' : 'category', 'kind': 'category', 'device_os': 'category', 'connection_type': 'category', 'ip_address': 'category', 'device_language': 'category'})

In [339]:
events_count = evts1.groupby('ref_hash').agg({'date': 'count'}).reset_index()
events_count.columns = ['ref_hash', 'cantidad de eventos']

events_count = target[['ref_hash']].merge(events_count, on = 'ref_hash', how = 'outer')
events_count['cantidad de eventos'] = events_count['cantidad de eventos'].fillna(0)
data =data.merge(events_count, on = 'ref_hash')
data=data[['ref_hash', 'installs_count', 'dia_semana_mean', 'hora_mean','cantidad de eventos','sc']]


In [340]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [341]:
data.head()

,ref_hash,installs_count,dia_semana_mean,hora_mean,cantidad de eventos,sc
0,1000169251625791246,0.0,0.0,0.0,3.0,259200.0
1,1000395625957344683,0.0,0.0,0.0,0.0,259200.0
2,1003027494996471685,2.0,2.0,21.0,22.0,259200.0
3,1006670001679961544,0.0,0.0,0.0,0.0,259200.0
4,1007573308966476713,0.0,0.0,0.0,0.0,259200.0


In [342]:
#Guardo la ventana con los features
data.to_csv(path_or_buf = 'data/features1_inst6.csv', index = False)
ts1= pd.read_csv( 'data/features1_inst6.csv')
ts1.head()

,ref_hash,installs_count,dia_semana_mean,hora_mean,cantidad de eventos,sc
0,1000169251625791246,0.0,0.0,0.0,3.0,259200.0
1,1000395625957344683,0.0,0.0,0.0,0.0,259200.0
2,1003027494996471685,2.0,2.0,21.0,22.0,259200.0
3,1006670001679961544,0.0,0.0,0.0,0.0,259200.0
4,1007573308966476713,0.0,0.0,0.0,0.0,259200.0


In [343]:
merged=ts1.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

# Armo los fetures del target

In [345]:
inst1 = pd.read_csv('data/installs_ventana7.csv', dtype = {'application_id': np.int16, 'ref_type': 'category', 'click_hash': 'category', 'device_country_code': 'category', 'device_brand': 'category', 'device_model': 'category', 'kind': 'category', 'device_language': 'category'})
inst1['created'] = pd.to_datetime(inst1['created'])
ids_list1=target['ref_hash'].unique()
inst1= inst1[inst1['ref_hash'].isin(ids_list1)]

In [346]:
install_count = inst1.groupby('ref_hash').agg({'created': 'count'}).reset_index()
install_count.columns = ['ref_hash', 'installs_count']
#hago un merge left para que darme solo con los ids del target
install_count = target[['ref_hash']].merge(install_count, on = 'ref_hash', how = 'left')
install_count['installs_count'] = install_count['installs_count'].fillna(0)

data = install_count
data['sc']=0

inst1['dia_semana'] = inst1['created'].dt.dayofweek
inst1['dia_semana']=inst1['dia_semana'] + 1
day_mean = inst1.groupby('ref_hash').agg({'dia_semana': 'mean'}).reset_index()
day_mean.columns = ['ref_hash', 'dia_semana_mean']

day_mean = target[['ref_hash']].merge(day_mean, on = 'ref_hash', how = 'left')
day_mean['dia_semana_mean'] = day_mean['dia_semana_mean'].fillna(0)

data =data.merge(day_mean, on = 'ref_hash')

inst1['hora']=inst1['created'].dt.hour
hour_mean = inst1.groupby('ref_hash').agg({'hora': 'mean'}).reset_index()
hour_mean.columns = ['ref_hash', 'hora_mean']
hour_mean['hora_mean'] = hour_mean['hora_mean'] +1
hour_mean = target[['ref_hash']].merge(hour_mean, on = 'ref_hash', how = 'left')
hour_mean['hora_mean'] = hour_mean['hora_mean'].fillna(0)

data =data.merge(hour_mean, on = 'ref_hash')

In [347]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [351]:
evts1 = pd.read_csv("data/events_ventana7.csv", dtype = {"event_id": np.int16, "application_id": np.int16, 'device_countrycode': 'category', 'device_os_version': 'category', 'device_brand': 'category', 'device_model': 'category', 'device_city': 'category', 'session_user_agent': 'category', 'trans_id': 'category', 'user_agent': 'category', 'carrier' : 'category', 'kind': 'category', 'device_os': 'category', 'connection_type': 'category', 'ip_address': 'category', 'device_language': 'category'})

In [352]:
events_count = evts1.groupby('ref_hash').agg({'date': 'count'}).reset_index()
events_count.columns = ['ref_hash', 'cantidad de eventos']

events_count = target[['ref_hash']].merge(events_count, on = 'ref_hash', how = 'left')
events_count['cantidad de eventos'] = events_count['cantidad de eventos'].fillna(0)
data =data.merge(events_count, on = 'ref_hash')
data=data[['ref_hash', 'installs_count', 'dia_semana_mean', 'hora_mean','cantidad de eventos','sc']]

In [353]:
merged=data.merge(target[['ref_hash']], on='ref_hash')
merged['ref_hash'].unique().size

4037

In [354]:
data.head()

,ref_hash,installs_count,dia_semana_mean,hora_mean,cantidad de eventos,sc
0,1000169251625791246,0.0,0.0,0.0,2.0,0
1,1000395625957344683,0.0,0.0,0.0,0.0,0
2,1003027494996471685,0.0,0.0,0.0,9.0,0
3,1006670001679961544,0.0,0.0,0.0,0.0,0
4,1007573308966476713,0.0,0.0,0.0,0.0,0


In [355]:
#Guardo la ventana con los features
data.to_csv(path_or_buf = 'data/features1_target_installs.csv', index = False)
ts1= pd.read_csv( 'data/features1_target_installs.csv')
ts1.head()

,ref_hash,installs_count,dia_semana_mean,hora_mean,cantidad de eventos,sc
0,1000169251625791246,0.0,0.0,0.0,2.0,0
1,1000395625957344683,0.0,0.0,0.0,0.0,0
2,1003027494996471685,0.0,0.0,0.0,9.0,0
3,1006670001679961544,0.0,0.0,0.0,0.0,0
4,1007573308966476713,0.0,0.0,0.0,0.0,0
